Chainlit used to create chat interface
Link: https://docs.chainlit.io/integrations/langchain

Useful Resources:
* Memory: https://www.youtube.com/watch?v=X550Zbz_ROE
* Llama RAG: https://www.youtube.com/watch?v=ypzmPwLH_Q4


In [ ]:
#!pip install huggingface_hub
#!pip install transformers
#!pip install langchain
#!pip install chainlit
#!pip install openai

In [2]:
#import openai
#import chainlit as cl
import os
from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from langchain.chains import ConversationChain
from dotenv import load_dotenv

load_dotenv('.env')

True

In [3]:
# model_id = "gpt2-medium" # 355M parameter
# model_id = "google/gemma-2b-it"
# model_id = "google/gemma-7b"
# model_id = "google/flan-ul2"
model_id = "meta-llama/Llama-2-7b-chat-hf"
temperature = 0.1

conv_model = HuggingFaceHub(huggingfacehub_api_token=os.environ['HUGGINGFACEHUB_API_TOKEN'],
                            repo_id=model_id,
                            model_kwargs={"temperature": temperature, "max_new_tokens":200}) # Temperature is randomness of model

c:\Anaconda3\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


GatedRepoError: 403 Client Error. (Request ID: Root=1-65e835d3-7798c0cf222757ab7b1c00f8;0eab0201-98bf-4465-b282-f070b13669c9)

Cannot access gated repo for url https://huggingface.co/api/models/meta-llama/Llama-2-7b-chat-hf.
Your request to access model meta-llama/Llama-2-7b-chat-hf is awaiting a review from the repo authors.

In [3]:
from langchain.chains.conversation.memory import ConversationBufferMemory, ConversationSummaryMemory, ConversationSummaryBufferMemory, ConversationKGMemory, ConversationEntityMemory 
from langchain.chains.conversation.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from pydantic import BaseModel
from typing import List, Dict, Any

conv_chain = ConversationChain(llm=conv_model,
                               verbose=True,
                               prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
                               memory=ConversationEntityMemory(llm = conv_model))

In [4]:
conv_chain.predict(input="Hi, I'm Julian. My TV is broken but is under warranty")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"Hi, I'm Julian. What can I help you with?"

In [5]:
conv_chain.predict(input="How can I get fix? Reference is A1241")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'I can help you with that. What is the reference number?'

In [7]:
conv_chain.predict(input="Reference number is A1241")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'I can help you with that. What is the reference number?'

In [8]:
# Useful commands
from pprint import pprint
pprint(conv_chain.memory.buffer)
pprint(conv_chain.memory.entity_store.store)
pprint(conv_chain.memory.entity_cache)

[HumanMessage(content="Hi, I'm Julian. My TV is broken but is under warranty", additional_kwargs={}),
 AIMessage(content="Hi, I'm Julian. What can I help you with?", additional_kwargs={}),
 HumanMessage(content='How can I get fix? Reference is A1241', additional_kwargs={}),
 AIMessage(content='I can help you with that. What is the reference number?', additional_kwargs={}),
 HumanMessage(content='Reference number is A1241', additional_kwargs={}),
 AIMessage(content='I can help you with that. What is the reference number?', additional_kwargs={})]
{'A1241': 'Reference number is A1241',
 'Julian': "Julian's reference number is A1241",
 'Julian.': "Julian's TV is broken but is under warranty."}
['Julian', 'A1241']


In [ ]:
template = """ The following is a customer service conversation between an human and AI friedly assistant.
If the AI doesn't know the answer, say only "I'm sorry, I don't have this information. Please contact sales.

Relevant Information:
{history}

Conversation:
Humna: {input}
AI: """

prompt = PromptTemplate(
    input_variables = ["history", "input"], template = template)

In [ ]:
template = """You are a customer service AI chat ready to response to the human user queries
{query}
"""
user_prompt = PromptTemplate(template=template, input_variables=['query'])